### Counterfactual analysis

In [ ]:
from utils_others import *
import json

In [ ]:
with open('config.json', 'r', encoding='utf-8') as f:
    config = json.load(f)

api_key = config["GEMINI_API_KEY"]


In [ ]:
gemini_hypers = {
    'temp': 0.2,  
    'alpha': 0.95,
    'beta': 0.3,
    'basic': [False],
    'settings': [SerializerSettings(base=10, prec=3, signed=True,half_bin_correction=True)],
}

model_hypers = {
    'gemini-1.0-pro': {'model': 'gemini-1.0-pro', **gemini_hypers},
}

model_predict_fns = {
    'gemini-1.0-pro': get_llmtime_predictions_data,
}


In [ ]:
datasets = get_datasets()  
datasets_tmp = get_memorization_datasets()
datasets.update(datasets_tmp)

In [ ]:
# The free version of Gemini always throws an error (security failure) when running the IstanbulTraffic dataset. While other datasets occasionally throw errors.
dataset_name = 'WineDataset'  
train, test = datasets[dataset_name]

num_samples = 10 

In [ ]:
def adding_noise(train, loc=0, steps=4, std=5):
    np.random.seed(42)
    noise = np.random.normal(0, std, steps)
    train[loc:(loc+steps)] += noise
    return train

In [ ]:
period = 12
std = 0.1*(train.max()-train.min())
num_iteration = int(len(train)/period)

out_counterfactual = {}
MSE =[]
MAE =[]
MAPE = []
R = []

for index in range(num_iteration):
    MSE_tmp =[]
    MAE_tmp =[]
    MAPE_tmp = []
    R_tmp = []
    train_tmp = copy.deepcopy(train)
    loc = index * period
    train_tmp = adding_noise(train_tmp, loc=loc, steps=period, std=std)
    if index > 0:
        time.sleep(60) 
    out_gemini_pro, out_gemini_pro_number = prediction_gemini(model_predict_fns, train, test, model_hypers, num_samples=num_samples, whether_blanket=False, dataset_name=dataset_name, genai_key=api_key)

    mse_amount = 0.0
    mae_amount = 0.0
    mape_amount = 0.0
    rsquare_amount = 0.0
    for i in range(num_samples):
        # seq_pred = out_gemini_pro[dataset_name]['samples'].iloc[i, :] 
        seq_pred = out_gemini_pro_number[dataset_name]['samples'].iloc[i, :] 
        
        mse = mean_squared_error(test, seq_pred)
        mae = mean_absolute_error(test, seq_pred)
        mape = metrics.mean_absolute_percentage_error(test, seq_pred)*100
        r2 = r2_score(test, seq_pred)
    
        MSE_tmp.append(mse)
        MAE_tmp.append(mae)
        MAPE_tmp.append(mape)
        R_tmp.append(r2)
    
    MSE.append(MSE_tmp)
    MAE.append(MAE_tmp)
    MAPE.append(MAPE_tmp)
    R.append(R_tmp)


In [ ]:
fig_counterfactual(R, metric_name='R^2', dataset_name='WineDataset')